In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt
from sklearn.linear_model import LinearRegression 
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
import os
os.chdir('../')

In [3]:
import yaml

with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

In [4]:
client = Socrata("data.seattle.gov", app_token=cfg['app_token'])

In [8]:
query = """
SELECT
  usageclass,
  checkoutmonth, 
  checkoutyear, 
  sum(checkouts) as monthly_checkouts
WHERE
  checkoutyear >= 2017
  and (materialtype = 'EBOOK' or materialtype = 'AUDIOBOOK' or materialtype = 'BOOK')
GROUP BY
  usageclass, checkoutmonth, checkoutyear
"""

In [9]:
result = client.get('tmmm-ytt6', query=query)

In [10]:
df = pd.DataFrame.from_records(result)

In [12]:
df['usageclass'] = df['usageclass'].astype('str')
df['checkoutmonth'] = df['checkoutmonth'].astype('int')
df['checkoutyear'] = df['checkoutyear'].astype('int')
df['monthly_checkouts'] = df['monthly_checkouts'].astype('int')
df['date'] = df.apply(lambda x: dt.datetime(year=x['checkoutyear'], month=x['checkoutmonth'], day=1), axis=1)

In [14]:
alt.Chart(df).mark_line().encode(
    x='date',
    y='monthly_checkouts',
    color='usageclass'
)

alt.Chart(...)

### What increase in digital book checkouts is due to covid-19?

In [16]:
digital = df[df['usageclass']=='Digital'].sort_values(by=['date'])

In [18]:
digital.head()

,usageclass,checkoutmonth,checkoutyear,monthly_checkouts,date
58,Digital,1,2017,185376,2017-01-01
31,Digital,2,2017,168370,2017-02-01
56,Digital,3,2017,186795,2017-03-01
69,Digital,4,2017,184500,2017-04-01
47,Digital,5,2017,187126,2017-05-01


In [19]:
digital['t'] = np.arange(1, len(digital) + 1)

In [20]:
def split_on_date(data, date):
    pre = data[data['date'] < date]
    post = data[data['date'] >= date]
    return pre, post

In [21]:
pre, post = split_on_date(digital, '2020-03-01')

In [22]:
fake_pre, fake_post = split_on_date(pre, '2019-12-01')

In [23]:
X_fake_pre = sm.add_constant(fake_pre[['t']])
X_fake_post = sm.add_constant(fake_post[['t']])
y_fake_pre = fake_pre[['monthly_checkouts']]

In [24]:
test_model = sm.OLS(y_fake_pre, X_fake_pre)

In [25]:
test_results = test_model.fit()

In [26]:
test_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      monthly_checkouts   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     543.4
Date:                Tue, 21 Jul 2020   Prob (F-statistic):           4.52e-22
Time:                        10:58:40   Log-Likelihood:                -369.43
No. Observations:                  35   AIC:                             742.9
Df Residuals:                      33   BIC:                             746.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.686e+05   3303.418     51.042      0.000    1.62e+05    1.75e+05
t           3730.8633    160.051     23.310      0.000    3405.237    4056.490
==============================================================================
Omnibus:                        1.099   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.577   Jarque-Bera (JB):                1.047
Skew:                           0.266   Prob(JB):                        0.593
Kurtosis:                       2.340   Cond. No.                         42.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
X_fake_post

,const,t
29,1.0,36
1,1.0,37
42,1.0,38


In [28]:
preds = test_results.predict(X_fake_post)

In [29]:
preds

29    302923.796639
1     306654.659944
42    310385.523249
dtype: float64

In [30]:
fake_post.loc[:, 'predicted'] = preds

/Users/carol/.pyenv/versions/bookenv/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/carol/.pyenv/versions/bookenv/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [31]:
fake_post.loc[:, 'diff'] = np.abs(fake_post['predicted'] - fake_post['monthly_checkouts'])

In [32]:
fake_post.loc[:, 'pct_err'] = fake_post['diff'] / fake_post['monthly_checkouts']

In [33]:
fake_post

,usageclass,checkoutmonth,checkoutyear,monthly_checkouts,date,t,predicted,diff,pct_err
29,Digital,12,2019,308176,2019-12-01,36,302923.796639,5252.203361,0.017043
1,Digital,1,2020,324650,2020-01-01,37,306654.659944,17995.340056,0.055430
42,Digital,2,2020,300871,2020-02-01,38,310385.523249,9514.523249,0.031623


In [34]:
X_pre = sm.add_constant(pre[['t']])
X_post = sm.add_constant(post[['t']])
y_pre = pre[['monthly_checkouts']]

In [35]:
model = sm.OLS(y_pre, X_pre)

In [36]:
results = model.fit()

In [37]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      monthly_checkouts   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.949
Method:                 Least Squares   F-statistic:                     682.6
Date:                Tue, 21 Jul 2020   Prob (F-statistic):           5.34e-25
Time:                        10:58:41   Log-Likelihood:                -402.04
No. Observations:                  38   AIC:                             808.1
Df Residuals:                      36   BIC:                             811.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.68e+05   3236.904     51.905      0.000    1.61e+05    1.75e+05
t           3780.2256    144.686     26.127      0.000    3486.788    4073.663
==============================================================================
Omnibus:                        1.973   Durbin-Watson:                   2.057
Prob(Omnibus):                  0.373   Jarque-Bera (JB):                1.390
Skew:                           0.234   Prob(JB):                        0.499
Kurtosis:                       2.188   Cond. No.                         45.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
preds = results.predict(X_post)

In [39]:
post.loc[:, 'predicted'] = preds

/Users/carol/.pyenv/versions/bookenv/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/carol/.pyenv/versions/bookenv/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [40]:
post.loc[:, 'diff'] = post['monthly_checkouts'] - post['predicted']
post.loc[:, 'pct_increase'] = post['diff'] / post['predicted']

In [41]:
post

,usageclass,checkoutmonth,checkoutyear,monthly_checkouts,date,t,predicted,diff,pct_increase
0,Digital,3,2020,321092,2020-03-01,39,315440.347084,5651.652916,0.017917
5,Digital,4,2020,354090,2020-04-01,40,319220.572710,34869.427290,0.109233
16,Digital,5,2020,383934,2020-05-01,41,323000.798337,60933.201663,0.188647
49,Digital,6,2020,380269,2020-06-01,42,326781.023963,53487.976037,0.163681


In [42]:
digital['predicted'] = results.predict(sm.add_constant(digital[['t']]))

In [43]:
digital['post_lockdown'] = digital['date'] >= dt.datetime(year=2020, month=3, day=1)

In [44]:
d = digital[digital['date'] >= dt.datetime(year=2019, month=1, day=1)]

regression_line = alt.Chart(d).mark_line().encode(
    x='date',
    y='predicted'
)

data_points = alt.Chart(d).mark_point().encode(
    x='date',
    y='monthly_checkouts',
    color='post_lockdown'
)

regression_line + data_points

alt.LayerChart(...)